In [37]:
import torch
import csv
import json
import os
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize 
from tqdm import tqdm
import math


In [38]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mrmrk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
class RemoveOnIterate:
    def __init__(self, data):
        self.data = data
        self.original_len = len(data)

    def __iter__(self):
        # This method should return the iterator object (in this case, self)
        return self

    def __next__(self):
        # Check if there are more elements in the data
        if self.data:
            # Get and remove the next element from the data
            result = self.data.pop(0)
            return result
        else:
            # No more elements, raise StopIteration
            raise StopIteration
    def __len__(self):
        return self.original_len

In [40]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [41]:
base_path = r"C:\Users\mrmrk\OneDrive\Documents\GitHub\recipe_generation\\"
original_cvs_name = r"RecipeNLG_dataset.csv"
prepeard_data_name = "prepeard_data.csv"
ingredient_vocab_name = "ingredient_vocab.csv"
recipie_vocab_name = "recipie_vocab.csv"

data_name = "data.csv"
data_tokanized_name = "data_tokanized.csv"

ing_count_name = "ingredient_count.pth"
rec_count_name = "recipie_count.pth"

ing_acum_dist_name = "ingredient_cumulative_distribution.pth"
rec_acum_dist_name = "recipie_cumulative_distribution.pth"

In [42]:
"""with open(base_path + original_cvs_name, 'r', newline='', encoding='utf-8') as input_csvfile:
    for data_len,row in enumerate(csv.reader(input_csvfile)):
        pass"""
data_len = 2231142
data_len

2231142

In [43]:
bads = ['\x00']

def fix_text(line, j):
    return j.join([r for r in json.loads(line) if not all([r.__contains__(bad) for bad in bads])])
if not os.path.exists(base_path + data_name):
    with open(base_path + original_cvs_name, 'r', newline='', encoding='utf-8') as input_csvfile:
        with open(base_path + data_name, 'w', newline='', encoding='utf-8') as output_csvfile:
            csv_writer = csv.writer(output_csvfile)
            csv_reader = csv.reader(input_csvfile)
            next(csv_reader, None)

            # Copy selected columns from the input file to the output file
            for row in (tqdm(csv_reader)):
                row = [fix_text(row[2], " * "),fix_text(row[3], " ")]
                csv_writer.writerow(row)
                


In [44]:

if not os.path.exists(base_path + data_tokanized_name):
    with open(base_path + data_name, 'r', newline='', encoding='utf-8') as input_csvfile:
        with open(base_path + data_tokanized_name, 'w', newline='', encoding='utf-8') as output_csvfile:
            csv_writer = csv.writer(output_csvfile)
            csv_reader = csv.reader(input_csvfile)
        
            for (ingrent_list, recipie) in tqdm(csv_reader, total=data_len):
                ingrent_tokalized = [w.lower() for w in word_tokenize(ingrent_list)]
                recipie_tokalized = [w.lower() for w in word_tokenize(recipie)]
                csv_writer.writerow((ingrent_tokalized, recipie_tokalized))

In [45]:
if not os.path.exists(base_path + ing_count_name):
    ingredients_count = Counter()
    recipies_count = Counter()

    with open(base_path + data_tokanized_name, 'r', newline='', encoding='utf-8') as input_csvfile:
        csv_reader = csv.reader(input_csvfile)
        for ingrent_tokalized, recipie_tokalized in tqdm(csv_reader, total=data_len):
            for word in eval(ingrent_tokalized):
                ingredients_count[word] += 1
            for word in eval(recipie_tokalized):
                recipies_count[word] += 1
    torch.save(ingredients_count, base_path + ing_count_name)
    torch.save(recipies_count, base_path + rec_count_name)
ingredients_count = torch.load(base_path + ing_count_name)
recipies_count =  torch.load(base_path + rec_count_name)

In [46]:
print(recipies_count.most_common(20))

[('.', 18681128), (',', 10728085), ('and', 9779961), ('the', 8766654), ('in', 5187219), ('to', 4966756), ('a', 4889946), ('with', 3344832), ('until', 2892925), ('add', 2730829), ('of', 2523757), ('minutes', 2393705), ('for', 2316425), ('or', 1644409), ('on', 1641420), ('into', 1608228), ('heat', 1585694), ('over', 1406082), (';', 1347366), ('mix', 1242485)]


In [47]:
len(ingredients_count)

114312

In [48]:
ingredients_count.most_common(10)

[('*', 17274871),
 ('1', 7409472),
 (',', 4281177),
 ('2', 3349649),
 ('.', 3174623),
 ('1/2', 2863843),
 ('cup', 2473045),
 ('c.', 2417218),
 ('(', 2308674),
 (')', 2306500)]

In [49]:
if not os.path.exists(base_path + ing_acum_dist_name):
    ing_acum_dist = []
    suum = 0
    s = sum(ingredients_count.values())
    for w, c in ingredients_count.most_common():
        suum += c
        ing_acum_dist.append(suum / s)
    rec_acum_dist = []
    suum = 0
    s = sum(recipies_count.values())
    for w, c in recipies_count.most_common():
        suum += c
        rec_acum_dist.append(suum / s)
    torch.save(ing_acum_dist, base_path + ing_acum_dist_name)
    torch.save(rec_acum_dist, base_path + rec_acum_dist_name)
ing_acum_dist = torch.load(base_path + ing_acum_dist_name)
rec_acum_dist =  torch.load(base_path + rec_acum_dist_name)
print(rec_acum_dist[:10])
print(rec_acum_dist[-10:])

[0.08287941257199803, 0.13047489946242902, 0.17386400437392316, 0.21275754087460752, 0.2357708012945515, 0.25780597067342664, 0.2795003700688374, 0.2943398225532924, 0.30717437540335835, 0.3192897842456176]
[0.9999999600712166, 0.9999999645077481, 0.9999999689442796, 0.999999973380811, 0.9999999778173426, 0.9999999822538741, 0.9999999866904056, 0.999999991126937, 0.9999999955634685, 1.0]


In [50]:
def find_index_larger_than(lst, x):
    for i, item in enumerate(lst):
        if item > x:
            return i
    return -1  # If no item is larger than x

In [51]:
ing_sum = sum(ingredients_count.values())
rec_sum = sum(recipies_count.values())
print(len(ingredients_count), len(recipies_count))
ing_prob = 0.994
rec_prob = 0.994
ingredient_words = ["<UKN>"] + [t[0] for t in ingredients_count.most_common(find_index_larger_than(ing_acum_dist, ing_prob))]
recipie_words = ["<UKN>"] + [t[0] for t in recipies_count.most_common(find_index_larger_than(rec_acum_dist, rec_prob))]
print(len(ingredient_words), len(recipie_words))
print(ingredients_count[ingredient_words[-1]], recipies_count[recipie_words[-1]])
ingredient_words[-1]

114312 259014
4767 8194
183 166


'etc.'

In [52]:
eval("['1', 'small', 'jar', 'chipped', 'beef', ',', 'cut', 'up', '*', '4', 'boned', 'chicken', 'breasts', '*', '1', 'can', 'cream', 'of', 'mushroom', 'soup', '*', '1', 'carton', 'sour', 'cream']")

['1',
 'small',
 'jar',
 'chipped',
 'beef',
 ',',
 'cut',
 'up',
 '*',
 '4',
 'boned',
 'chicken',
 'breasts',
 '*',
 '1',
 'can',
 'cream',
 'of',
 'mushroom',
 'soup',
 '*',
 '1',
 'carton',
 'sour',
 'cream']

In [53]:
ing_word_to_id = dict(zip(ingredient_words, list(range(len(ingredient_words)))))
rec_word_to_id = dict(zip(recipie_words, list(range(len(recipie_words)))))
if not os.path.exists(base_path + prepeard_data_name):
    with open(base_path + prepeard_data_name, 'w', newline='', encoding='utf-8') as output_csvfile:
        with open(base_path + data_tokanized_name, 'r', newline='', encoding='utf-8') as input_csvfile:
            csv_reader = csv.reader(input_csvfile)
            csv_writer = csv.writer(output_csvfile)
            a = next(csv_reader)
            csv_writer.writerow(["X","Y"])

            for ing, rec in tqdm(csv_reader, total=data_len):
                ing_ids = [ing_word_to_id.get(i,0) for i in eval(ing)]
                rec_ids = [rec_word_to_id.get(r,0) for r in eval(rec)]
                csv_writer.writerow((ing_ids, rec_ids))

 36%|███▋      | 808905/2231142 [01:59<03:30, 6755.72it/s]

In [ ]:

with open(base_path + ingredient_vocab_name, 'w', newline='', encoding='utf-8') as output_csvfile:
    csv_writer = csv.writer(output_csvfile)

    csv_writer.writerow(["words"])

    # Copy selected columns from the input file to the output file
    for row in ingredient_words:
        csv_writer.writerow([row])

In [ ]:

with open(base_path + recipie_vocab_name, 'w', newline='', encoding='utf-8') as output_csvfile:
    csv_writer = csv.writer(output_csvfile)

    csv_writer.writerow(["wors"])

    # Copy selected columns from the input file to the output file
    for row in recipie_words:
        csv_writer.writerow([row])